In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import re
import pandas as pd
import numpy as np
import requests
import re
from itertools import groupby


def Get_detail_stagaires_ma(URL):
    html = requests.get(URL).text
    bs_ = BeautifulSoup(html,'html.parser')
    Company_name = bs_.find('h4',{'class':'inline'}).get_text()
    Profil = bs_.findAll('div',{'class':'col-sm-4 col-xs-6 m-b-sm'})
    Profil_recherches = []
    for i in Profil:
       #Profil_recherche=Profil_recherche + i.strong.get_text()+' | '
       Profil_recherches.append(i.strong.get_text())
    Type = Profil_recherches[0]
    Duree_du_stage = Profil_recherches[2]
    Convention = Profil_recherches[3]
    Prime_de_stage = Profil_recherches[4]
    Types_de_stage = Profil_recherches[5]
    Niveau_etude = bs_.find('div',{'class':'col-sm-4 col-xs-6'}).strong.get_text()
    Type_de_formation = bs_.find('div',{'class':'col-sm-8 col-xs-6'}).strong.get_text()
    Descriptif_de_la_mission = bs_.find('div',{'class':'well well-sm m-b-none'})
    Resultat = ''
    for i in Descriptif_de_la_mission:
      i=re.sub('<p>', ' ', str(i))
      i=re.sub("</p>", ' ', i)
      i=re.sub('</ul>', ' ', i)
      i=re.sub('<ul>', ' ', i)
      i=re.sub('<li>', ' ', i)
      i=re.sub('</li>', ' ', i)
      i=re.sub('<u>', ' ', i)
      i=re.sub('</u>', ' ', i)
      Resultat = Resultat+ i
    return Resultat,Company_name,Duree_du_stage,Convention,Prime_de_stage,Types_de_stage,Niveau_etude,Type_de_formation
    

      
    
        
    #print(description_entreprise)
    #i = bs_.findAll('p')
    #DESCRIPTION_ENTREPRISE=i[0].get_text()
    #CONTEXTE_MISSION=i[1].get_text()
    #PROFIL_RECHERCHE=i[2].get_text()
    #return(DESCRIPTION_ENTREPRISE,CONTEXTE_MISSION,PROFIL_RECHERCHE)

def Get_stagaires_ma(PageNumber):  
  html = requests.get('https://www.stagiaires.ma/offres-stages/{}'.format(PageNumber)).text
  bs = BeautifulSoup(html,'html.parser')
  Offres = bs.findAll('div',{'class':'offer-container p-xxs m-b-xs bg-muted'})
  data = {'Titre': [],'Company_name':[],'stage-depart':[],'Duree_du_stage':[],'Convention':[],'Types_de_stage':[],'Prime_de_stage':[],'Niveau_etude':[],'Type_de_formation':[],'Descriptif_de_la_mission':[]}
  for offre in Offres:
    Titre = offre.find('strong').get_text()
    data['Titre'].append(Titre)

    stage_depart = offre.find('small', {'data-placement':'top'}).get_text()
    data['stage-depart'].append(stage_depart)

    link = offre.a['href']
    Resultat,Company_name,Duree_du_stage,Convention,Prime_de_stage,Types_de_stage,Niveau_etude,Type_de_formation = Get_detail_stagaires_ma(str(link))
    data['Company_name'].append(Company_name)
    data['Duree_du_stage'].append(Duree_du_stage)
    data['Convention'].append(Convention)
    data['Prime_de_stage'].append(Prime_de_stage)
    data['Types_de_stage'].append(Types_de_stage)
    data['Niveau_etude'].append(Niveau_etude)
    data['Type_de_formation'].append(Type_de_formation)
    
    data['Descriptif_de_la_mission'].append(Resultat)



  return pd.DataFrame(data)





pages = ['','1','2','3','4','5','6','7']
appended_data = []
for page in pages :    
    data =Get_stagaires_ma(page)
    appended_data.append(data)
appended_data = pd.concat(appended_data)


In [2]:
appended_data.to_csv('StagairesMa.csv', index=False, encoding='utf-8')   

In [3]:
appended_data.to_excel('StagairesMa.xlsx', index=False, encoding='utf-8')